In [1]:
import uuid
from googleads import adwords
import sys

In [2]:
from csv import reader
import csv
import pandas as pd
import numpy as np

In [3]:
AUTH_FILE_PATH = '/home/tim_su/ai_optimizer/opt/ai_optimizer/googleads.yaml'
client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#client_2 = ad_manager.AdManagerClient.LoadFromStorage('/home/tim_su/ai_optimizer/opt/ai_optimizer/googleads.yaml')

In [4]:
CUSTOMER_SERVICE_ID = '3637290511'
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)

In [ ]:
# Display which field is able to query for certain report type
REPORT_TYPE = 'CAMPAIGN_PERFORMANCE_REPORT'

def get_report_fields(client, report_type):
    # Initialize appropriate service.
    report_definition_service = client.GetService(
        'ReportDefinitionService', version='v201809')

    # Get report fields.
    fields = report_definition_service.getReportFields(report_type)

    # Display results.
    print('Report type "%s" contains the following fields:' % report_type)
    for field in fields:
        print(' - %s(%s)' % (field['fieldName'], field['fieldType']))
        if 'enumValues' in field:
            print('  := [%s]' % ', '.join(field['enumValues']))

if __name__ == '__main__':
    # Initialize client object.
    adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
    get_report_fields(adwords_client, REPORT_TYPE)

In [ ]:
CAMPAIGN_FIELDS = ['ExternalCustomerId','CampaignId','AdvertisingChannelType', 'CampaignStatus',
                   'BiddingStrategyType','Amount','StartDate','EndDate','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion']
class DatePreset:
    today = 'TODAY'
    yesterday = 'YESTERDAY'
    lifetime = 'ALL_TIME'

def get_campaign_insights(client, campaign_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    report = {
        'reportName': 'CAMPAIGN_PERFORMANCE_REPORT',
#         'dateRangeType': 'CUSTOM_DATE',
        'dateRangeType': date_preset,
        'reportType': 'CAMPAIGN_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': CAMPAIGN_FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
            'predicates': [
                {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
                }
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(CAMPAIGN_FIELDS)),
            columns=CAMPAIGN_FIELDS
        )
        df = df.append(df_temp)
    df[df.columns.difference(['CampaignStatus', 'AdvertisingChannelType', 'BiddingStrategyType', 'StartDate', 'EndDate'])] = df[df.columns.difference(
        ['CampaignStatus', 'AdvertisingChannelType', 'BiddingStrategyType', 'StartDate', 'EndDate'])].apply(pd.to_numeric, errors='coerce')
    df[df.columns.difference(['ExternalCustomerId', 'CampaignId', 'AdvertisingChannelType', 'CampaignStatus', 'BiddingStrategyType', 'StartDate', 'EndDate', 'Impressions', 'Clicks', 'Conversions'])] = df[df.columns.difference(
        ['ExternalCustomerId', 'CampaignId', 'CampaignStatus', 'AdvertisingChannelType', 'BiddingStrategyType', 'StartDate', 'EndDate', 'Impressions', 'Clicks', 'Conversions'])].div(1000000)
    df[['StartDate','EndDate']] = df[['StartDate','EndDate']].apply( pd.to_datetime, errors='coerce' )
    return df

if __name__ == '__main__':
    # Initialize client object.
    client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
    campaign_insights = get_campaign_insights(client, campaign_id=campaign_id)


In [323]:
ADGROUP_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus',
                  'CpmBid','CpvBid', 'CpcBid', 'TargetCpa', 'BiddingStrategyType','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion']
# ADGROUP_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId','ConversionCategoryName']
def get_adgroup_insights(client, campaign_id=None, adgroup_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    if campaign_id is not None and adgroup_id is None:
        operand = {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
        }
    elif adgroup_id is not None:
        operand = {
                    'field': 'AdGroupId',
                    'operator': 'EQUALS',
                    'values':[adgroup_id]
        }
    else:
        print('get_adgroup_insights: Missing arguments campaign_id or adgroup_id.' )
        operand = None
        return
    report = {
        'reportName': 'ADGROUP_PERFORMANCE_REPORT',
#         'dateRangeType': 'CUSTOM_DATE',
        'dateRangeType': date_preset,
        'reportType': 'ADGROUP_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': ADGROUP_FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
            'predicates': [
                operand
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(ADGROUP_FIELDS)),
            columns=ADGROUP_FIELDS
        )
        df = df.append(df_temp)
    df[df.columns.difference(['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])] = df[df.columns.difference(
        ['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType'])].apply(pd.to_numeric, errors='coerce')
    df[df.columns.difference( ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'] )] = df[df.columns.difference(
        ['ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType', 'Conversions'])].div(1000000)

    return df.reset_index(drop=True)

if __name__ == '__main__':
    # Initialize client object.
    client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#     adgroup_insights = get_adgroup_insights(client)
    adgroup_insights = get_adgroup_insights(client, campaign_id=campaign_id)

NameError: name 'ADGROUP_FIELDS' is not defined

## Retrieve Age Range Performance Report

In [324]:
AGE_RANGE_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupId','Criteria', 'Id', 'AdGroupStatus',
                  'CpmBid', 'CpcBid', 'BiddingStrategyType','Cost',
                   'AverageCost','Impressions', 'Clicks','Conversions',
                   'AverageCpc','CostPerConversion']
# AGE_RANGE_FIELDS = ['ExternalCustomerId','CampaignId', 'AdGroupType', 'AdGroupId','ConversionCategoryName']
def get_age_range_insights(client, campaign_id=None, adgroup_id=None, date_preset=None):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    if campaign_id is not None and adgroup_id is None:
        operand = {
                    'field': 'CampaignId',
                    'operator': 'EQUALS',
                    'values':[campaign_id]
        }
    elif adgroup_id is not None:
        operand = {
                    'field': 'AdGroupId',
                    'operator': 'EQUALS',
                    'values':[adgroup_id]
        }
    else:
        print('get_age_range_insights: Missing arguments campaign_id or adgroup_id.' )
        operand = None
        return
    report = {
        'reportName': 'AGE_RANGE_PERFORMANCE_REPORT',
        'dateRangeType': date_preset,
        'reportType': 'AGE_RANGE_PERFORMANCE_REPORT',
        'downloadFormat': 'CSV',
        'selector': {
            'fields': AGE_RANGE_FIELDS,
            'predicates': [
                operand
            ]
        }
    }
    # Print out the report as a string
    csv = report_downloader.DownloadReportAsString(  
        report, skip_report_header=True, skip_column_header=True,   
        skip_report_summary=True, include_zero_impressions=True,client_customer_id=CUSTOMER_SERVICE_ID)
    csv_list = csv.split('\n')[:-1]
    df = pd.DataFrame()
    for lil_csv in csv_list:
        df_temp = pd.DataFrame(
            data=np.array(lil_csv.split(',')).reshape(1,len(AGE_RANGE_FIELDS)),
            columns=AGE_RANGE_FIELDS
        )
        df = df.append(df_temp)
    df[df.columns.difference(['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType', 'Criteria'])] = df[df.columns.difference(
        ['AdGroupType', 'AdGroupStatus', 'BiddingStrategyType', 'Criteria'])].apply(pd.to_numeric, errors='coerce')
    df[df.columns.difference( ['ExternalCustomerId', 'CampaignId','Criteria', 'Id', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType','Impressions', 'Clicks', 'Conversions'] )] = df[df.columns.difference(
        ['ExternalCustomerId', 'CampaignId', 'Criteria', 'Id', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'BiddingStrategyType','Impressions', 'Clicks', 'Conversions'])].div(1000000)

    return df.reset_index(drop=True)

# if __name__ == '__main__':
# Initialize client object.
client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
#     adgroup_insights = get_age_range_insights(client)
adgroup_insights = get_age_range_insights(client, campaign_id=1747836670, adgroup_id=68524904775)

In [325]:
adgroup_insights

,ExternalCustomerId,CampaignId,AdGroupId,Criteria,Id,AdGroupStatus,CpmBid,CpcBid,BiddingStrategyType,Cost,AverageCost,Impressions,Clicks,Conversions,AverageCpc,CostPerConversion
0,3637290511,1747836670,68524904775,45-54,503004,enabled,NaN,4.0,cpc,730.107691,3.349118,120704,218,3.0,3.349118,243.369230
1,3637290511,1747836670,68524904775,55-64,503005,enabled,NaN,4.0,cpc,344.631714,2.591216,53271,133,5.0,2.591216,68.926343
2,3637290511,1747836670,68524904775,25-34,503002,enabled,NaN,4.0,cpc,142.175938,3.231271,33607,44,0.0,3.231271,0.000000
3,3637290511,1747836670,68524904775,65 or more,503006,enabled,NaN,4.0,cpc,666.542294,2.593550,86322,257,4.0,2.593550,166.635574
4,3637290511,1747836670,68524904775,35-44,503003,enabled,NaN,4.0,cpc,223.367736,3.384360,46368,66,0.0,3.384360,0.000000
5,3637290511,1747836670,68524904775,18-24,503001,enabled,NaN,4.0,cpc,0.000000,0.000000,0,0,0.0,0.000000,0.000000
6,3637290511,1747836670,68524904775,Undetermined,503999,enabled,NaN,4.0,cpc,1564.560144,3.245975,328620,482,6.0,3.245975,260.760024


In [ ]:
# URL
URL_FIELDS_LIST = [
    'ExternalCustomerId', 'CampaignId', 'AdGroupId', 'AdGroupStatus', 'Cost', 'AverageCost',
    'Impressions', 'Clicks', 'Conversions', 'AverageCpc', 'CostPerConversion', 'DisplayName', ]

# CRITERIA,
CRITERIA_FIELDS_LIST = [
    'ExternalCustomerId', 'CampaignId', 'AdGroupId', 'Criteria', 'Id', 'AdGroupStatus', 'CpmBid', 'CpcBid',
    'Cost', 'AverageCost', 'Impressions', 'Clicks', 'Conversions', 'AverageCpc', 'CostPerConversion']

# AUDIENCE, AGE_RANGE, DISPLAY_KEYWORD
BIDDABLE_FIELDS_LIST = [
    'ExternalCustomerId', 'CampaignId', 'AdGroupId', 'Criteria', 'Id', 'AdGroupStatus', 'CpmBid', 'CpcBid',
    'BiddingStrategyType', 'Cost', 'AverageCost', 'Impressions', 'Clicks', 'Conversions', 'AverageCpc', 'CostPerConversion']

# ADGROUP
ADGROUP_FIELDS_LIST = [
    'ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'CpmBid', 'CpvBid', 'CpcBid', 'TargetCpa',
    'BiddingStrategyType', 'Cost', 'AverageCost', 'Impressions', 'Clicks', 'Conversions', 'AverageCpc', 'CostPerConversion']

NON_NUMERIC_FIELDS_LIST = [
    'Ad group type', 'Ad group state', 'Bid Strategy Type', 'Keyword', 'Age Range', 'Audience', 'Keyword / Placement', 'Criteria Display Name']

NUMERIC_FIELDS_LIST = [
    'Max. CPM', 'Max. CPV', 'Max. CPC', 'Default max. CPC', 'Target CPA', 'Cost', 'Avg. Cost', 'Avg. CPC', 'Cost / conv.']

FIELD_INDEX = {
    'ADGROUP': ADGROUP_FIELDS_LIST,
    'URL': URL_FIELDS_LIST,
    'CRITERIA': CRITERIA_FIELDS_LIST,
    'AUDIENCE': BIDDABLE_FIELDS_LIST,
    'AGE_RANGE': BIDDABLE_FIELDS_LIST,
    'DISPLAY_KEYWORD': BIDDABLE_FIELDS_LIST,
}

In [542]:
ADGROUP_FIELDS_LIST  = [
    'customer_id', 'campaign_id', 'channel_type', 'adgroup_id', 'status', 'cpm_bid', 'cpv_bid', 'cpc_bid', 'cpa_bid', 'bidding_type', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']
URL_FIELDS_LIST      = [
    'customer_id', 'campaign_id', 'adgroup_id', 'status', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion', 'url_display_name']
CRITERIA_FIELDS_LIST = [
    'customer_id', 'campaign_id', 'adgroup_id', 'keyword_placement', 'keyword_id', 'status', 'cpm_bid', 'cpc_bid', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']
AUDIENCE_FIELDS_LIST = [
    'customer_id', 'campaign_id', 'adgroup_id', 'audience', 'criterion_id', 'status', 'cpm_bid', 'cpc_bid', 'bidding_type', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']
AGE_RANGE_FIELDS_LIST = [
    'customer_id', 'campaign_id', 'adgroup_id', 'age_range', 'criterion_id', 'status', 'cpm_bid', 'cpc_bid', 'bidding_type', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']
BIDDABLE_FIELDS_LIST = [
    'customer_id', 'campaign_id', 'adgroup_id', 'keyword', 'keyword_id', 'status', 'cpm_bid', 'cpc_bid', 'bidding_type', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']

COLUMN_INDEX = {
    'ADGROUP': ADGROUP_FIELDS_LIST,
    'URL': URL_FIELDS_LIST,
    'CRITERIA': CRITERIA_FIELDS_LIST,
    'AUDIENCE': AUDIENCE_FIELDS_LIST,
    'AGE_RANGE': AGE_RANGE_FIELDS_LIST,
    'DISPLAY_KEYWORD': BIDDABLE_FIELDS_LIST,
}
DATABASE_INDEX = {
    'ADGROUP': 'adgroup_insights',
    'URL': 'url_insights',
    'CRITERIA': 'criteria_insights',
    'AUDIENCE': 'audience_insights',
    'AGE_RANGE': 'age_range_insights',
    'DISPLAY_KEYWORD': 'display_keyword_insights',
}

In [543]:





def get_performance_insights(client, campaign_id=None, adgroup_id=None, date_preset=None, performance_type='ADGROUP'):
    client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    if campaign_id is not None and adgroup_id is None:
        operand = {
            'field': 'CampaignId',
            'operator': 'EQUALS',
            'values': [campaign_id]
        }
    elif adgroup_id is not None:
        operand = {
            'field': 'AdGroupId',
            'operator': 'EQUALS',
            'values': [adgroup_id]
        }
    else:
        print('get_audience_insights: Missing arguments campaign_id or adgroup_id.')
        operand = None
        return
    report = {
        'downloadFormat': 'CSV',
        'reportName': performance_type+'_PERFORMANCE_REPORT',
        'reportType': performance_type+'_PERFORMANCE_REPORT',
        'dateRangeType': date_preset,
        'selector': {
            'fields': FIELD_INDEX[performance_type],
            'predicates': [operand]
        }
    }
    # Print out the report as a string
    with open(performance_type+'.csv', 'wb') as output_file:
        report_downloader.DownloadReport(
            report, output=output_file, skip_report_header=True, skip_column_header=False,
            skip_report_summary=True, include_zero_impressions=False, client_customer_id=CUSTOMER_SERVICE_ID)
    with open(performance_type+'.csv')as csv_file:
        df = pd.read_csv(csv_file, sep=",", quotechar='"')
        
        df[df.columns.difference( NON_NUMERIC_FIELDS_LIST )] = df[df.columns.difference( NON_NUMERIC_FIELDS_LIST )].apply(pd.to_numeric, errors='coerce')
        
        df[df.columns.intersection( NUMERIC_FIELDS_LIST )] = df[df.columns.intersection( NUMERIC_FIELDS_LIST )].div(1000000)
        
        df.columns = COLUMN_INDEX[performance_type]
        gdn_db.into_table( df, DATABASE_INDEX[performance_type] )
        return df.sort_values(by=['impressions'], ascending=False).reset_index(drop=True)


if __name__ == '__main__':
    # Initialize client object.
    # URL
    # CRITERIA
    # AUDIENCE, AGE_RANGE, DISPLAY_KEYWORD
    from ai_optimizer.codes import gdn_db
    client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
    display_keyword_insights = get_performance_insights(
        client, campaign_id=1747836664, adgroup_id=68524904535, performance_type='DISPLAY_KEYWORD')

In [511]:
from ai_optimizer.codes import gdn_db

In [538]:
gdn_db.into_table(display_keyword_insights, "display_keyword_insights")

In [ ]:
from googleads import adwords
CUSTOMER_SERVICE_ID = '3637290511'
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
AD_GROUP_ID = 68524904055
CPC_BID_MICRO_AMOUNT = '4000000'


def update_adgroup_bid(client, ad_group_id, bid_micro_amount=None):
    # Initialize appropriate service.
    ad_group_service = client.GetService('AdGroupService', version='v201809')
    
    # Construct operations and update an ad group.
    operations = [{
        'operator': 'SET',
        'operand': {
            'id': ad_group_id,
            'status': 'ENABLED'
        }
    }]
  
    if bid_micro_amount:
        operations[0]['operand']['biddingStrategyConfiguration'] = {
            'bids': [{
                'xsi_type': 'CpcBid',
                'bid': {
                    'microAmount': bid_micro_amount,
                }
            }]
        }
  
    ad_groups = ad_group_service.mutate(operations)
    return ad_groups

if __name__ == '__main__':
    # Initialize client object.
    adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
    adwords_client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
    update_adgroup_bid(adwords_client, AD_GROUP_ID, CPC_BID_MICRO_AMOUNT)

In [ ]:

adwords_client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
adwords_client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
update_adgroup_bid(adwords_client, AD_GROUP_ID, CPC_BID_MICRO_AMOUNT)

In [168]:
def retrieve_campaign_params(client, campaign_id):
    _FIELDS = ['CampaignId', 'Name']
    selector= [{
        'fields': _FIELDS,
    #             'dateRange': {'min': '20190301','max': '20190401'},
        'predicates': [
            {
                'field': 'CampaignId',
                'operator': 'EQUALS',
                'values':[campaign_id]
            }
        ]
    }]
    campaign_service = client.GetService('CampaignService', version='v201809')
    campaign_params = campaign_service.get(selector)
    return campaign_params

In [170]:
retrieve_campaign_params(client, campaign_id)['entries'][0]['name']

'華友旅行_4月_CPC$8_3/28-4/27_CT_AI'

In [186]:
def retrieve_ad_group_params(client, ad_group_id):
    _FIELDS = ['AdGroupId', 'Name', 'CpcBid']
    selector= [{
        'fields': _FIELDS,
    #             'dateRange': {'min': '20190301','max': '20190401'},
        'predicates': [
            {
                'field': 'AdGroupId',
                'operator': 'EQUALS',
                'values':[ad_group_id]
            }
        ]
    }]
    ad_group_service = client.GetService('AdGroupService', version='v201809')
    ad_group_params = ad_group_service.get(selector)
    return ad_group_params

In [188]:
retrieve_ad_group_params(client, ad_group_id)['entries'][0]['biddingStrategyConfiguration']

{
    'biddingStrategyId': None,
    'biddingStrategyName': None,
    'biddingStrategyType': 'MANUAL_CPC',
    'biddingStrategySource': None,
    'biddingScheme': None,
    'bids': [
        {
            'Bids.Type': 'CpcBid',
            'bid': {
                'ComparableValue.Type': 'Money',
                'microAmount': 3000000
            },
            'cpcBidSource': 'ADGROUP'
        }
    ],
    'targetRoasOverride': None
}

In [175]:
CUSTOMER_SERVICE_ID = '3637290511'
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)
def retrieve_ad_group_criterion(client, ad_group_id):
    _FIELDS = ['AdGroupId', 'CriteriaType', 'UserInterestId', 'UserInterestName', 'UserListId', 'LabelIds']
    selector= [{
        'fields': _FIELDS,
    #             'dateRange': {'min': '20190301','max': '20190401'},
        'predicates': [
            {
                'field': 'AdGroupId',
                'operator': 'EQUALS',
                'values':[ad_group_id]
            }
        ]
    }]
    ad_group_service = client.GetService('AdGroupCriterionService', version='v201809')
    ad_group_criterion = ad_group_service.get(selector)
    return ad_group_criterion


In [27]:
import copy
###
new_adgroup_id = 67966950646
###
def make_adgroup_criterion(new_adgroup_id = None, old_ad_group_criterion=None):
    if new_adgroup_id is None and old_ad_group_criterion is None:
        raise ValueError('new_adgroup_id and old_ad_group_criterion is Required.')
    biddable_criterion['adGroupId'] = new_adgroup_id
    negative_criterion['adGroupId'] = new_adgroup_id

    biddable_ad_group_criteria_list = list()
    negative_ad_group_criteria_list = list()
    for entry in old_ad_group_criterion['entries']:
        criterion = dict()

        if 'AgeRange' in entry['criterion']['Criterion.Type']:
    #         print(entry['criterion']['id'], entry['criterion']['Criterion.Type'])
            criterion['xsi_type'] = entry['criterion']['Criterion.Type']
            criterion['id'] = entry['criterion']['id']

        elif 'Placement'  in entry['criterion']['Criterion.Type']:
    #         print(entry['criterion']['id'], entry['criterion']['url'])
            criterion['xsi_type'] = entry['criterion']['Criterion.Type']
            criterion['id'] = entry['criterion']['id']
            criterion['url'] = entry['criterion']['url']

        elif 'Keyword'  in entry['criterion']['Criterion.Type']:
    #         print(entry['criterion']['id'], entry['criterion']['text'])
            criterion['xsi_type'] = entry['criterion']['Criterion.Type']
            criterion['id'] = entry['criterion']['id']
            criterion['matchType'] = entry['criterion']['matchType']
            criterion['text'] = entry['criterion']['text']
    #         print(criterion)

        elif 'CriterionUserInterest'  in entry['criterion']['Criterion.Type']:
    #         print(entry['criterion']['id'], entry['criterion'])
            criterion['xsi_type'] = entry['criterion']['Criterion.Type']
            criterion['userInterestId'] = entry['criterion']['userInterestId']

        if criterion:
            if 'BiddableAdGroupCriterion' in entry['AdGroupCriterion.Type']:
                biddable_criterion['xsi_type'] = entry['AdGroupCriterion.Type']
                biddable_criterion['criterion'] = criterion
                biddable_ad_group_criteria_list.append(copy.deepcopy(biddable_criterion))

            elif 'NegativeAdGroupCriterion' in entry['AdGroupCriterion.Type']:
                negative_criterion['xsi_type'] = entry['AdGroupCriterion.Type']
                negative_criterion['criterion'] = criterion
                negative_ad_group_criteria_list.append(copy.deepcopy(negative_criterion))
    return biddable_ad_group_criteria_list, negative_ad_group_criteria_list
 

In [26]:
def assign_criterion_to_ad_group(client, ad_group_id, ad_group_criterion):
    '''
    an example of addong 4 type of criterion to new adgroup,
    if trying to exclude certain criterions, 
    change 'xsi_type' from 'BiddableAdGroupCriterion' to 'NegativeAdGroupCriterion' in ad_group_criteria object
    '''
    # Initialize appropriate service.
    ad_group_criterion_service = client.GetService(
        'AdGroupCriterionService', version='v201809')
    # Create the ad group criteria.
    ad_group_criteria = ad_group_criterion
  
    # Create operations.
    operations = []
    for criterion in ad_group_criteria:
        operations.append({
            'operator': 'ADD',
            'operand': criterion
        })
  
    response = ad_group_criterion_service.mutate(operations)
    return response

In [25]:
new_ad_group_id = ad_groups['value'][0]['id']

In [192]:
import uuid
campaign_id = 1777436620
def make_ad_group(campaign_id, ad_group_id):
    ad_group_params = retrieve_ad_group_params(client, ad_group_id)
    name = ad_group_params['entries'][0]['name']
    bids = ad_group_params['entries'][0]['biddingStrategyConfiguration']
    operations = [{
        'operator': 'ADD',
        'operand': {
            'campaignId': campaign_id,
            'name':  'Mutant #%s' % uuid.uuid4(),
            'status': 'PAUSED',
            'biddingStrategyConfiguration': bids
        }
    }]
    ad_group_service = client.GetService('AdGroupService', version='v201809')
    ad_groups = ad_group_service.mutate(operations)
    return ad_groups

In [161]:
make_ad_group(campaign_id, ad_group_id)

{
    'ListReturnValue.Type': 'AdGroupReturnValue',
    'value': [
        {
            'id': 70785567553,
            'campaignId': 1777436620,
            'campaignName': '華友旅行_4月_CPC$8_3/28-4/27_CT_AI',
            'name': 'TEST_3 5011ff38-24f9-4007-afbc-700f8c8d6814',
            'status': 'PAUSED',
            'settings': [
                {
                    'Setting.Type': 'TargetingSetting',
                    'details': [
                        {
                            'criterionTypeGroup': 'GENDER',
                            'targetAll': True
                        },
                        {
                            'criterionTypeGroup': 'AGE_RANGE',
                            'targetAll': True
                        },
                        {
                            'criterionTypeGroup': 'PARENT',
                            'targetAll': True
                        },
                        {
                            'criterionTypeGroup': 'INCO

## Make New AdGroup With Criterion From Old AdGroup

In [30]:
old_ad_group_id = 67966950646
old_ad_group_criterion = retrieve_ad_group_criterion(client, old_ad_group_id)
ad_group_params = make_ad_group(campaign_id, old_ad_group_id)
new_ad_group_id = ad_group_params['value'][0]['id']
biddable_criterion, negative_criterion = make_adgroup_criterion(new_ad_group_id, old_ad_group_criterion)
new_ad_group_criterion = biddable_criterion + negative_criterion
result = assign_criterion_to_ad_group(client, new_ad_group_id, new_ad_group_criterion)

In [32]:
retrieve_ad_group_params(client, new_ad_group_id)

{
    'totalNumEntries': 19,
    'Page.Type': 'AdGroupCriterionPage',
    'entries': [
        {
            'adGroupId': 69550439176,
            'criterionUse': 'NEGATIVE',
            'criterion': {
                'id': 503001,
                'type': 'AGE_RANGE',
                'Criterion.Type': 'AgeRange',
                'ageRangeType': 'AGE_RANGE_18_24'
            },
            'labels': [],
            'forwardCompatibilityMap': [],
            'baseCampaignId': None,
            'baseAdGroupId': None,
            'AdGroupCriterion.Type': 'NegativeAdGroupCriterion'
        },
        {
            'adGroupId': 69550439176,
            'criterionUse': 'BIDDABLE',
            'criterion': {
                'id': 503002,
                'type': 'AGE_RANGE',
                'Criterion.Type': 'AgeRange',
                'ageRangeType': 'AGE_RANGE_25_34'
            },
            'labels': [],
            'forwardCompatibilityMap': [],
            'baseCampaignId': None,
       

In [82]:
_FIELDS = ['Id', 'AdType', 'HeadlinePart1', 'HeadlinePart2', 'DisplayUrl', 'CreativeFinalUrls', 'Description', 'Url']
selector= [{
    'fields': _FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
    'predicates': [
        {
            'field': 'Id',
            'operator': 'EQUALS',
            'values':[343427883002]
        }
#         {
#             'field': 'AdGroupId',
#             'operator': 'EQUALS',
#             'values':[67966950646]
#         }
    ]
}]
ad_service = client.GetService('AdService', version='v201809')
ad_params = ad_service.get(selector)

In [83]:
ad_params

{
    'totalNumEntries': 1,
    'Page.Type': 'AdPage',
    'entries': [
        {
            'id': 343427883002,
            'url': None,
            'displayUrl': 'www.mitravel.com.tw',
            'finalUrls': [
                'http://www.mitravel.com.tw/tour_feature.php?id=271'
            ],
            'finalMobileUrls': [],
            'finalAppUrls': [],
            'trackingUrlTemplate': None,
            'finalUrlSuffix': None,
            'urlCustomParameters': None,
            'urlData': [],
            'automated': None,
            'type': 'IMAGE_AD',
            'devicePreference': None,
            'systemManagedEntitySource': None,
            'Ad.Type': 'ImageAd',
            'image': {
                'mediaId': None,
                'type': 'IMAGE',
                'referenceId': 8731361575950090240,
                'dimensions': [
                    {
                        'key': 'FULL',
                        'value': {
                            'width': 3

In [69]:
_FIELDS = ['AdGroupId', 'CriterionId']
selector= [{
    'fields': _FIELDS,
#             'dateRange': {'min': '20190301','max': '20190401'},
    'predicates': [
        {
            'field': 'CampaignId',
#             'field': 'AdGroupId',
            'operator': 'EQUALS',
#             'values':['IMAGE_AD']
            'values':[67966950646]
        }
    ]
}]
ad_service = client.GetService('AdParamService', version='v201809')
ad_params = ad_service.get(selector)

In [146]:
ad_group_id = 71763883120
def retrive_ad_params(ad_group_id):
    ad_params_list = []
    ad_criteria = dict()
    _FIELDS = ['AdGroupId', 'HeadlinePart1', 'HeadlinePart2', 'DisplayUrl', 'CreativeFinalUrls', 'Description', 'Url']
    selector= [{
        'fields': _FIELDS,
        'predicates': [
            {
                'field': 'AdGroupId',
                'operator': 'EQUALS',
                'values':[ad_group_id]
            }
        ]
    }]
    ad_service = client.GetService('AdGroupAdService', version='v201809')
    ad_params = ad_service.get(selector)
    for entry in ad_params['entries']:
        ad_criteria['id'] = entry['ad']['id']
        ad_params_list.append(copy.deepcopy(ad_criteria))
    return ad_params_list

In [151]:
def assign_ad_to_ad_group(ad_group_id, ad_params_list):
    operations = []
    for ad_params in ad_params_list:
        operations.append({
            'operator': 'ADD',
            'operand': {
                'adGroupId': ad_group_id,
                'status': 'ENABLED',
                'ad': ad_params
            }
        })
    ad_group_service = client.GetService('AdGroupAdService', version='v201809')
    ad_groups = ad_group_service.mutate(operations)
    return ad_groups

In [20]:
ad_group_service = client.GetService('AdGroupService', version='v201809')

# Construct operations and add ad groups.
operations = [{
    'operator': 'ADD',
    'operand': {
        'campaignId': 1777436620,
        'name': 'TEST_2',
        'status': 'PAUSED',
        'biddingStrategyConfiguration': {
            'bids': [
                {
                    'xsi_type': 'CpcBid',
                    'bid': {
                        'microAmount': '1000000'
                    },
                }
            ]
        },
    }
}]
ad_groups = ad_group_service.mutate(operations)

In [ ]:
client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
ad_group_service = client.GetService('AdGroupService', version='v201809')

# Construct operations and update an ad group.
operations = [{
    'operator': 'SET',
    'operand': {
        'id': ad_group_id,
        'status': 'ACTIVE'
    }
}]

if bid_micro_amount:
    operations[0]['operand']['biddingStrategyConfiguration'] = {
      'bids': [{
          'xsi_type': 'CpcBid',
          'bid': {
              'microAmount': bid_micro_amount,
          }
      }]
    }

ad_groups = ad_group_service.mutate(operations)